In [ ]:
def construire_sql(select_label, where_template, concepts_valeurs):

    select_clause = f"SELECT {select_label} FROM films"


    where_clause = where_template

    for concept, valeur in concepts_valeurs:
        if concept == "acteur":
            clause_acteur = f"(acteur1 = '{valeur}' OR acteur2 = '{valeur}' OR acteur3 = '{valeur}')"
            where_clause = where_clause.replace("<val>", clause_acteur, 1)
        elif concept == "annee":
            where_clause = where_clause.replace("<annee>", valeur, 1)
        else:
            where_clause = where_clause.replace("<val>", f"'{valeur}'", 1)

    requete_sql = f"{select_clause} WHERE {where_clause};"
    return requete_sql


In [ ]:
##python

import pandas as pd
import sqlite3
import re
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline

def nettoyer_where(clause):
    clause = re.sub(r"'[^']*'", "'<val>'", clause)
    clause = re.sub(r"\d{4}", "<annee>", clause)
    return clause.lower().strip()

def extraire_labels(sql):
    select = re.search(r'select (.+?) from', sql, re.IGNORECASE)
    where = re.search(r'where (.+)', sql, re.IGNORECASE)
    select_label = select.group(1).strip().lower() if select else "UNKNOWN"
    where_label = nettoyer_where(where.group(1)) if where else "NONE"
    return select_label, where_label

def analyse_lexicale(requete, lexique):
    concepts_valeurs = []
    deja_vus = set()
    correspondances = []
    for concept, valeurs in lexique.items():
        for valeur in valeurs:
            if valeur.lower() in requete.lower():
                match = re.search(re.escape(valeur), requete, flags=re.IGNORECASE)
                if match:
                    correspondances.append((match.start(), valeur, concept))
    correspondances.sort()
    for _, valeur, concept in correspondances:
        if valeur.lower() not in deja_vus:
            deja_vus.add(valeur.lower())
            concept_label = "acteur" if concept.startswith("acteur") else concept
            concepts_valeurs.append((concept_label, valeur))
    return concepts_valeurs

def generer_where_sql(concepts_valeurs):
    clauses = []
    for concept, valeur in concepts_valeurs:
        if concept == "acteur":
            clause = f"(acteur1 = '{valeur}' OR acteur2 = '{valeur}' OR acteur3 = '{valeur}')"
        elif concept == "annee":
            clause = f"annee = {valeur}"
        else:
            clause = f"{concept} = '{valeur}'"
        clauses.append(clause)
    return " AND ".join(clauses)

def executer_requete_sqlite(select_clause, where_clause, conn):
    sql = f"SELECT {select_clause} FROM films WHERE {where_clause};"
    try:
        result = pd.read_sql_query(sql, conn)
        return sql, result
    except Exception as e:
        return sql, f"Erreur d'exécution : {e}"

def main():
    # Charger les données d'entraînement
    with open("queries_french_para.json", encoding="utf-8") as f:
        data = json.load(f)

    select_X, select_y, where_X, where_y = [], [], [], []

    for entry in data:
        sql = entry.get("sql", "")
        select_label, where_label = extraire_labels(sql)
        phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
        for phrase in phrases:
            select_X.append(phrase)
            select_y.append(select_label)
            where_X.append(phrase)
            where_y.append(where_label)

    pipeline_select = Pipeline([("vectorizer", CountVectorizer()), ("perceptron", Perceptron())])
    pipeline_select.fit(select_X, select_y)

    # Charger les données du CSV
    df = pd.read_csv("base_films_500.csv")
    conn = sqlite3.connect(":memory:")
    df.to_sql("films", conn, index=False, if_exists="replace")

    valeurs_concepts = {
        'titre': df['titre'].dropna().unique().tolist(),
        'realisateur': df['realisateur'].dropna().unique().tolist(),
        'annee': df['annee'].dropna().astype(str).unique().tolist(),
        'genre': df['genre'].dropna().unique().tolist(),
        'acteur1': df['acteur1'].dropna().unique().tolist(),
        'acteur2': df['acteur2'].dropna().unique().tolist(),
        'acteur3': df['acteur3'].dropna().unique().tolist()
    }

    while True:
        requete_naturelle = input("\nEntrez votre requête (ou tapez 'exit' pour quitter) :\n> ")
        if requete_naturelle.lower() == "exit":
            print("À bientôt !")
            break

        concepts_valeurs = analyse_lexicale(requete_naturelle, valeurs_concepts)
        select_pred = pipeline_select.predict([requete_naturelle])[0]
        where_clause = generer_where_sql(concepts_valeurs)
        requete_sql, resultat = executer_requete_sqlite(select_pred, where_clause, conn)

        print("\n--- Requête SQL générée ---")
        print(requete_sql)
        print("\n--- Résultat ---")
        if isinstance(resultat, pd.DataFrame):
            print(resultat.head(10))
        else:
            print(resultat)

if __name__ == "__main__":
    main()



In [ ]:
select_pred = "titre"
where_pred = "(acteur1 = '<val>' OR acteur2 = '<val>' OR acteur3 = '<val>') AND (acteur1 = '<val>' OR acteur2 = '<val>' OR acteur3 = '<val>')"
concepts_vals = [('acteur', 'Meryl Streep'), ('acteur', 'Hugh Jackman')]

sql = construire_sql(select_pred, where_pred, concepts_vals)
print(sql)


SELECT titre FROM films WHERE (acteur1 = '(acteur1 = 'Meryl Streep' OR acteur2 = 'Meryl Streep' OR acteur3 = 'Meryl Streep')' OR acteur2 = '(acteur1 = 'Hugh Jackman' OR acteur2 = 'Hugh Jackman' OR acteur3 = 'Hugh Jackman')' OR acteur3 = '<val>') AND (acteur1 = '<val>' OR acteur2 = '<val>' OR acteur3 = '<val>');


In [ ]:
import pandas as pd
import sqlite3
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline


with open("queries_french_para.json", encoding="utf-8") as f:
    data = json.load(f)

def nettoyer_where(clause):
    clause = re.sub(r"'[^']*'", "'<val>'", clause)
    clause = re.sub(r"\d{4}", "<annee>", clause)
    return clause.lower().strip()

def extraire_labels(sql):
    select = re.search(r'select (.+?) from', sql, re.IGNORECASE)
    where = re.search(r'where (.+)', sql, re.IGNORECASE)
    select_label = select.group(1).strip().lower() if select else "UNKNOWN"
    where_label = nettoyer_where(where.group(1)) if where else "NONE"
    return select_label, where_label

select_X, select_y, where_X, where_y = [], [], [], []
for entry in data:
    sql = entry.get("sql", "")
    select_label, where_label = extraire_labels(sql)
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for phrase in phrases:
        select_X.append(phrase)
        select_y.append(select_label)
        where_X.append(phrase)
        where_y.append(where_label)


pipeline_select = Pipeline([("vectorizer", CountVectorizer()), ("perceptron", Perceptron())])
pipeline_select.fit(select_X, select_y)


df = pd.read_csv("base_films_500.csv")
conn = sqlite3.connect(":memory:")
df.to_sql("films", conn, index=False, if_exists="replace")


valeurs_concepts = {
    'titre': df['titre'].dropna().unique().tolist(),
    'realisateur': df['realisateur'].dropna().unique().tolist(),
    'annee': df['annee'].dropna().astype(str).unique().tolist(),
    'genre': df['genre'].dropna().unique().tolist(),
    'acteur1': df['acteur1'].dropna().unique().tolist(),
    'acteur2': df['acteur2'].dropna().unique().tolist(),
    'acteur3': df['acteur3'].dropna().unique().tolist()
}


def analyse_lexicale(requete, lexique):
    concepts_valeurs = []
    deja_vus = set()
    correspondances = []
    for concept, valeurs in lexique.items():
        for valeur in valeurs:
            if valeur.lower() in requete.lower():
                match = re.search(re.escape(valeur), requete, flags=re.IGNORECASE)
                if match:
                    correspondances.append((match.start(), valeur, concept))
    correspondances.sort()
    for _, valeur, concept in correspondances:
        if valeur.lower() not in deja_vus:
            deja_vus.add(valeur.lower())
            concept_label = "acteur" if concept.startswith("acteur") else concept
            concepts_valeurs.append((concept_label, valeur))
    return concepts_valeurs


def generer_where_sql(concepts_valeurs):
    clauses = []
    annees = []

    for concept, valeur in concepts_valeurs:
        if concept == "annee":
            annees.append(valeur)
        elif concept == "acteur":
            val = valeur
            clause = f"((acteur1 = '{val}' or acteur2 = '{val}' or acteur3 = '{val}'))"
            clauses.append(clause)
        else:
            clause = f"{concept} = '{valeur}'"
            clauses.append(clause)


    if len(annees) == 2:
        an1, an2 = sorted(annees)
        clauses.append(f"annee between {an1} and {an2}")
    elif len(annees) == 1:
        clauses.append(f"annee = {annees[0]}")

    return " AND ".join(clauses)



def executer_requete_sqlite(select_clause, where_clause):
    sql = f"SELECT {select_clause} FROM films WHERE {where_clause};"
    try:
        result = pd.read_sql_query(sql, conn)
        return sql, result
    except Exception as e:
        return sql, f"Erreur d'exécution : {e}"


requete_naturelle = "Je souhaiterais connaître les films où Meryl Streep et Hugh Jackman ont un rôle."
concepts_valeurs = analyse_lexicale(requete_naturelle, valeurs_concepts)
select_pred = pipeline_select.predict([requete_naturelle])[0]
where_clause = generer_where_sql(concepts_valeurs)
requete_sql, resultat = executer_requete_sqlite(select_pred, where_clause)

print("\n--- Requête SQL générée ---")
print(requete_sql)
print("\n--- Résultat ---")
print(resultat.head(10) if isinstance(resultat, pd.DataFrame) else resultat)



--- Requête SQL générée ---
SELECT titre FROM films WHERE ((acteur1 = 'Meryl Streep' or acteur2 = 'Meryl Streep' or acteur3 = 'Meryl Streep')) AND ((acteur1 = 'Hugh Jackman' or acteur2 = 'Hugh Jackman' or acteur3 = 'Hugh Jackman'));

--- Résultat ---
                        titre
0  The Frozen Mission Project
1      The Frozen Memory Code
2        The Lost Secret Game


#Evaluation :    

#Evaluation quantitative :

In [ ]:
import json
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron

# === Charger les fichiers nécessaires
with open("queries_french_para.json", encoding="utf-8") as f:
    train_data = json.load(f)

with open("queries_french_para_eval.json", encoding="utf-8") as f:
    eval_data = json.load(f)

# === Fonction d’extraction de concepts depuis le SQL attendu
def extraire_concepts_from_sql(sql):
    sql = sql.lower()
    concepts = []
    if "titre" in sql: concepts.append("titre")
    if "realisateur" in sql: concepts.append("realisateur")
    if "annee" in sql: concepts.append("annee")
    if "genre" in sql: concepts.append("genre")
    if "acteur1" in sql or "acteur2" in sql or "acteur3" in sql:
        concepts.append("acteur")
    return list(set(concepts))

# === Analyse lexicale déjà existante
def analyse_lexicale(requete, lexique):
    concepts_valeurs = []
    deja_vus = set()
    correspondances = []
    for concept, valeurs in lexique.items():
        for valeur in valeurs:
            if valeur.lower() in requete.lower():
                match = re.search(re.escape(valeur), requete, flags=re.IGNORECASE)
                if match:
                    correspondances.append((match.start(), valeur, concept))
    correspondances.sort()
    for _, valeur, concept in correspondances:
        if valeur.lower() not in deja_vus:
            deja_vus.add(valeur.lower())
            concept_label = "acteur" if concept.startswith("acteur") else concept
            concepts_valeurs.append((concept_label, valeur))
    return concepts_valeurs

# === Préparation du lexique depuis le CSV
import pandas as pd
df = pd.read_csv("base_films_500.csv")

valeurs_concepts = {
    'titre': df['titre'].dropna().unique().tolist(),
    'realisateur': df['realisateur'].dropna().unique().tolist(),
    'annee': df['annee'].dropna().astype(str).unique().tolist(),
    'genre': df['genre'].dropna().unique().tolist(),
    'acteur1': df['acteur1'].dropna().unique().tolist(),
    'acteur2': df['acteur2'].dropna().unique().tolist(),
    'acteur3': df['acteur3'].dropna().unique().tolist()
}

# === Entraînement des modèles de classification SELECT/WHERE
def nettoyer_where(clause):
    clause = re.sub(r"'[^']*'", "'<val>'", clause)
    clause = re.sub(r"\d{4}", "<annee>", clause)
    return clause.lower().strip()

def extraire_labels(sql):
    select = re.search(r'select (.+?) from', sql, re.IGNORECASE)
    where = re.search(r'where (.+)', sql, re.IGNORECASE)
    select_label = select.group(1).strip().lower() if select else "UNKNOWN"
    where_label = nettoyer_where(where.group(1)) if where else "NONE"
    return select_label, where_label

select_X, select_y, where_X, where_y = [], [], [], []

for entry in train_data:
    sql = entry["sql"]
    select_label, where_label = extraire_labels(sql)
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for phrase in phrases:
        select_X.append(phrase)
        select_y.append(select_label)
        where_X.append(phrase)
        where_y.append(where_label)

# === Modèles Perceptron
pipeline_select = Pipeline([("vec", CountVectorizer()), ("clf", Perceptron())])
pipeline_where = Pipeline([("vec", CountVectorizer()), ("clf", Perceptron())])
pipeline_select.fit(select_X, select_y)
pipeline_where.fit(where_X, where_y)

# === Évaluation 1 : Concepts / Valeurs
print("\n🔍 Évaluation 1 : Extraction des concepts/valeurs\n")
total = 0
correct = 0

for entry in eval_data:
    sql_attendu = entry["sql"]
    concepts_attendus = set(extraire_concepts_from_sql(sql_attendu))
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for phrase in phrases:
        concepts_extraits = set([c for c, _ in analyse_lexicale(phrase, valeurs_concepts)])
        if concepts_attendus == concepts_extraits:
            correct += 1
        else:
            print(f"❌ Phrase : {phrase}")
            print(f"Attendu : {concepts_attendus}")
            print(f"Extrait : {concepts_extraits}\n")
        total += 1

print(f"\n✅ Précision concept/valeurs : {correct}/{total} = {correct / total * 100:.2f}%")

# === Évaluation 2 : SELECT + WHERE
print("\n🧠 Évaluation 2 : Intention SELECT et WHERE\n")
select_correct, where_correct = 0, 0
total_intent = 0

for entry in eval_data:
    sql_attendu = entry["sql"]
    select_gold, where_gold = extraire_labels(sql_attendu)
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for phrase in phrases:
        select_pred = pipeline_select.predict([phrase])[0]
        where_pred = pipeline_where.predict([phrase])[0]
        if select_pred.strip().lower() == select_gold:
            select_correct += 1
        else:
            print(f"❌ SELECT - {phrase}")
            print(f"Gold : {select_gold} | Pred : {select_pred}")
        if where_pred.strip().lower() == where_gold:
            where_correct += 1
        else:
            print(f"❌ WHERE - {phrase}")
            print(f"Gold : {where_gold} | Pred : {where_pred}")
        total_intent += 1

print(f"\n✅ Précision SELECT : {select_correct}/{total_intent} = {select_correct / total_intent * 100:.2f}%")
print(f"✅ Précision WHERE  : {where_correct}/{total_intent} = {where_correct / total_intent * 100:.2f}%")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Extrait : {'titre'}

❌ Phrase : Pouvez-vous me donner l'année des films intitulés The Last Empire Story ?
Attendu : {'titre', 'annee'}
Extrait : {'titre'}

❌ Phrase : Est-ce que vous pourriez me dire en quelle année sont sortis les films The Last Empire Story ?
Attendu : {'titre', 'annee'}
Extrait : {'titre'}

❌ Phrase : J'aimerais connaître l'année de sortie des films appelés The Last Empire Story.
Attendu : {'titre', 'annee'}
Extrait : {'titre'}

❌ Phrase : Pourriez-vous m'indiquer l'année des films portant le titre The Last Empire Story ?
Attendu : {'titre', 'annee'}
Extrait : {'titre'}

❌ Phrase : Donnez-moi, s'il vous plaît, l'année des films qui s'appellent The Last Empire Story.
Attendu : {'titre', 'annee'}
Extrait : {'titre'}

❌ Phrase : Veuillez me montrer l'année des films dont le titre est The Silent Journey Protocol.
Attendu : {'titre', 'annee'}
Extrait : {'titre'}

❌ Phrase : Pouvez-vous me dire 

In [ ]:
import json
import pandas as pd
import sqlite3
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron


with open("queries_french_para.json", encoding="utf-8") as f:
    data_train = json.load(f)

with open("queries_french_para_eval.json", encoding="utf-8") as f:
    data_eval = json.load(f)


def normaliser_sql(chaine):
    chaine = chaine.lower()
    chaine = re.sub(r'\s+', ' ', chaine)
    chaine = re.sub(r'\s*=\s*', '=', chaine)
    chaine = re.sub(r'\s*between\s*', ' between ', chaine)
    chaine = chaine.strip().rstrip(';')
    return chaine


def nettoyer_where(clause):
    clause = re.sub(r"'[^']*'", "'<val>'", clause)
    clause = re.sub(r"\d{4}", "<annee>", clause)
    return clause.lower().strip()

def extraire_labels(sql):
    select = re.search(r'select (.+?) from', sql, re.IGNORECASE)
    where = re.search(r'where (.+)', sql, re.IGNORECASE)
    select_label = select.group(1).strip().lower() if select else "UNKNOWN"
    where_label = nettoyer_where(where.group(1)) if where else "NONE"
    return select_label, where_label


select_X, select_y = [], []
for entry in data_train:
    select_label, _ = extraire_labels(entry["sql"])
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for phrase in phrases:
        select_X.append(phrase)
        select_y.append(select_label)

pipeline_select = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("perceptron", Perceptron())
])
pipeline_select.fit(select_X, select_y)


df = pd.read_csv("base_films_500.csv")

valeurs_concepts = {
    'titre': df['titre'].dropna().unique().tolist(),
    'realisateur': df['realisateur'].dropna().unique().tolist(),
    'annee': df['annee'].dropna().astype(str).unique().tolist(),
    'genre': df['genre'].dropna().unique().tolist(),
    'acteur1': df['acteur1'].dropna().unique().tolist(),
    'acteur2': df['acteur2'].dropna().unique().tolist(),
    'acteur3': df['acteur3'].dropna().unique().tolist()
}


def analyse_lexicale(requete, lexique):
    concepts_valeurs = []
    deja_vus = set()
    correspondances = []
    for concept, valeurs in lexique.items():
        for valeur in valeurs:
            if valeur.lower() in requete.lower():
                match = re.search(re.escape(valeur), requete, flags=re.IGNORECASE)
                if match:
                    correspondances.append((match.start(), valeur, concept))
    correspondances.sort()
    for _, valeur, concept in correspondances:
        if valeur.lower() not in deja_vus:
            deja_vus.add(valeur.lower())
            concept_label = "acteur" if concept.startswith("acteur") else concept
            concepts_valeurs.append((concept_label, valeur))
    return concepts_valeurs


def generer_where_sql(concepts_valeurs):
    clauses = []
    annees = []

    for concept, valeur in concepts_valeurs:
        if concept == "annee":
            annees.append(valeur)
        elif concept == "acteur":
            clause = f"((acteur1 = '{valeur}' or acteur2 = '{valeur}' or acteur3 = '{valeur}'))"
            clauses.append(clause)
        else:
            clause = f"{concept} = '{valeur}'"
            clauses.append(clause)

    if len(annees) == 2:
        an1, an2 = sorted(annees)
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee between {an1} and {an2}")
    elif len(annees) == 1:
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee = {annees[0]}")

    return " AND ".join(clauses)


total = 0
correct = 0

for entry in data_eval:
    expected_sql = normaliser_sql(entry["sql"])
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]

    for phrase in phrases:
        total += 1
        concepts_valeurs = analyse_lexicale(phrase, valeurs_concepts)
        select_pred = pipeline_select.predict([phrase])[0]
        where_clause = generer_where_sql(concepts_valeurs)
        sql_gen = f"SELECT {select_pred} FROM films WHERE {where_clause};"
        sql_gen_norm = normaliser_sql(sql_gen)

        print("\n📝 Phrase :", phrase)
        print("✅ Générée :", sql_gen)
        print("🎯 Attendue :", entry["sql"])

        if sql_gen_norm == expected_sql:
            correct += 1
            print("✔️ CORRECT")
        else:
            print("❌ INCORRECT")


print(f"\n✅ Évaluation terminée : {correct} / {total} corrects")
print(f"🎯 Précision : {correct / total * 100:.2f}%")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
🎯 Attendue : SELECT titre FROM films WHERE (acteur1 = 'Anne Hathaway' OR acteur2 = 'Anne Hathaway' OR acteur3 = 'Anne Hathaway') AND genre = 'Drame';
❌ INCORRECT

📝 Phrase : Pourriez-vous me montrer les films d'Anne Hathaway classés dans le genre Drame ?
✅ Générée : SELECT titre FROM films WHERE ((acteur1 = 'Anne Hathaway' or acteur2 = 'Anne Hathaway' or acteur3 = 'Anne Hathaway')) AND genre = 'Drame';
🎯 Attendue : SELECT titre FROM films WHERE (acteur1 = 'Anne Hathaway' OR acteur2 = 'Anne Hathaway' OR acteur3 = 'Anne Hathaway') AND genre = 'Drame';
❌ INCORRECT

📝 Phrase : J'aimerais connaître les titres des films d'Anne Hathaway qui sont des Drames, s'il vous plaît.
✅ Générée : SELECT titre FROM films WHERE ((acteur1 = 'Anne Hathaway' or acteur2 = 'Anne Hathaway' or acteur3 = 'Anne Hathaway')) AND genre = 'Drame';
🎯 Attendue : SELECT titre FROM films WHERE (acteur1 = 'Anne Hathaway' OR acteur2 = 'Anne Hathaw

In [ ]:
import json
import pandas as pd
import sqlite3
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron

# Charger corpus d'entraînement
with open("queries_french_para.json", encoding="utf-8") as f:
    data_train = json.load(f)

# Charger corpus d'évaluation (avec valeurs)
with open("queries_french_para_eval_values.json", encoding="utf-8") as f:
    data_eval = json.load(f)

# Prétraitement du SQL
def extraire_labels(sql):
    select = re.search(r'select (.+?) from', sql, re.IGNORECASE)
    select_label = select.group(1).strip().lower() if select else "UNKNOWN"
    return select_label

# Entraînement modèle SELECT
select_X, select_y = [], []
for entry in data_train:
    select_label = extraire_labels(entry["sql"])
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for phrase in phrases:
        select_X.append(phrase)
        select_y.append(select_label)

pipeline_select = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("perceptron", Perceptron())
])
pipeline_select.fit(select_X, select_y)

# Charger la base films
df = pd.read_csv("base_films_500.csv")
conn = sqlite3.connect(":memory:")
df.to_sql("films", conn, index=False, if_exists="replace")

# Création du lexique
valeurs_concepts = {
    'titre': df['titre'].dropna().unique().tolist(),
    'realisateur': df['realisateur'].dropna().unique().tolist(),
    'annee': df['annee'].dropna().astype(str).unique().tolist(),
    'genre': df['genre'].dropna().unique().tolist(),
    'acteur1': df['acteur1'].dropna().unique().tolist(),
    'acteur2': df['acteur2'].dropna().unique().tolist(),
    'acteur3': df['acteur3'].dropna().unique().tolist()
}

# Analyse lexicale
def analyse_lexicale(requete, lexique):
    concepts_valeurs = []
    deja_vus = set()
    correspondances = []
    for concept, valeurs in lexique.items():
        for valeur in valeurs:
            if valeur.lower() in requete.lower():
                match = re.search(re.escape(valeur), requete, flags=re.IGNORECASE)
                if match:
                    correspondances.append((match.start(), valeur, concept))
    correspondances.sort()
    for _, valeur, concept in correspondances:
        if valeur.lower() not in deja_vus:
            deja_vus.add(valeur.lower())
            concept_label = "acteur" if concept.startswith("acteur") else concept
            concepts_valeurs.append((concept_label, valeur))
    return concepts_valeurs

# WHERE avec BETWEEN et double parenthèses
def generer_where_sql(concepts_valeurs):
    clauses = []
    annees = []
    for concept, valeur in concepts_valeurs:
        if concept == "annee":
            annees.append(valeur)
        elif concept == "acteur":
            clause = f"((acteur1 = '{valeur}' or acteur2 = '{valeur}' or acteur3 = '{valeur}'))"
            clauses.append(clause)
        else:
            clause = f"{concept} = '{valeur}'"
            clauses.append(clause)
    if len(annees) == 2:
        an1, an2 = sorted(annees)
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee between {an1} and {an2}")
    elif len(annees) == 1:
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee = {annees[0]}")
    return " AND ".join(clauses)

# Fonction d'exécution et comparaison
def extraire_reponse(select, where):
    try:
        sql = f"SELECT {select} FROM films WHERE {where};"
        result = pd.read_sql_query(sql, conn)
        if not result.empty:
            val = str(result.iloc[0, 0])
            return val.strip()
        return None
    except:
        return None

# ÉVALUATION
total = 0
correct = 0

for entry in data_eval:
    phrase = entry["query"]
    valeur_attendue = entry["value"].strip()

    concepts_valeurs = analyse_lexicale(phrase, valeurs_concepts)
    select_pred = pipeline_select.predict([phrase])[0]
    where_clause = generer_where_sql(concepts_valeurs)
    valeur_obtenue = extraire_reponse(select_pred, where_clause)

    print("\n🔎 Query :", phrase)
    print("🧠 SQL → SELECT", select_pred, "WHERE", where_clause)
    print("🎯 Attendu :", valeur_attendue)
    print("✅ Obtenu  :", valeur_obtenue)

    total += 1
    if valeur_obtenue and valeur_obtenue.strip().lower() == valeur_attendue.lower():
        correct += 1
        print("✔️ Correct")
    else:
        print("❌ Incorrect")

# Résultat global
print(f"\n📊 Évaluation terminée : {correct} / {total} corrects")
print(f"🎯 Précision : {correct / total * 100:.2f}%")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
🔎 Query : Pourriez-vous m'indiquer qui a réalisé The Infinite Shadow Affair ?
🧠 SQL → SELECT realisateur WHERE titre = 'The Infinite Shadow Affair'
🎯 Attendu : Bong Joon-ho
✅ Obtenu  : Bong Joon-ho
✔️ Correct

🔎 Query : Je souhaiterais savoir qui est le réalisateur du film The Silent Shadow Game.
🧠 SQL → SELECT realisateur WHERE titre = 'The Silent Shadow Game'
🎯 Attendu : Stanley Kubrick
✅ Obtenu  : Stanley Kubrick
✔️ Correct

🔎 Query : J'aimerais savoir l'année des films intitulés The Infinite Shadow Affair.
🧠 SQL → SELECT annee WHERE titre = 'The Infinite Shadow Affair'
🎯 Attendu : 2011
✅ Obtenu  : 2011
✔️ Correct

🔎 Query : Est-ce que vous pourriez m'indiquer le réalisateur des films intitulés The Golden Shadow Code ?
🧠 SQL → SELECT realisateur WHERE titre = 'The Golden Shadow Code'
🎯 Attendu : James Cameron
✅ Obtenu  : James Cameron
✔️ Correct

🔎 Query : Je voudrais connaître le réalisateur des films int

##Evaluation qualitative :

In [ ]:
import pandas as pd
import sqlite3
import re
import json
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron

# Charger données d'entraînement
with open("queries_french_para.json", encoding="utf-8") as f:
    data_train = json.load(f)

# Charger base films
df = pd.read_csv("base_films_500.csv")
conn = sqlite3.connect(":memory:")
df.to_sql("films", conn, index=False, if_exists="replace")

# Lexique
valeurs_concepts = {
    'titre': df['titre'].dropna().unique().tolist(),
    'realisateur': df['realisateur'].dropna().unique().tolist(),
    'annee': df['annee'].dropna().astype(str).unique().tolist(),
    'genre': df['genre'].dropna().unique().tolist(),
    'acteur1': df['acteur1'].dropna().unique().tolist(),
    'acteur2': df['acteur2'].dropna().unique().tolist(),
    'acteur3': df['acteur3'].dropna().unique().tolist()
}

# Analyse lexicale
def analyse_lexicale(requete, lexique):
    concepts_valeurs = []
    deja_vus = set()
    correspondances = []
    for concept, valeurs in lexique.items():
        for valeur in valeurs:
            if valeur.lower() in requete.lower():
                match = re.search(re.escape(valeur), requete, flags=re.IGNORECASE)
                if match:
                    correspondances.append((match.start(), valeur, concept))
    correspondances.sort()
    for _, valeur, concept in correspondances:
        if valeur.lower() not in deja_vus:
            deja_vus.add(valeur.lower())
            concept_label = "acteur" if concept.startswith("acteur") else concept
            concepts_valeurs.append((concept_label, valeur))
    return concepts_valeurs


def generer_where_sql(concepts_valeurs):
    clauses = []
    annees = []
    for concept, valeur in concepts_valeurs:
        if concept == "annee":
            annees.append(valeur)
        elif concept == "acteur":
            clause = f"((acteur1 = '{valeur}' or acteur2 = '{valeur}' or acteur3 = '{valeur}'))"
            clauses.append(clause)
        else:
            clause = f"{concept} = '{valeur}'"
            clauses.append(clause)
    if len(annees) == 2:
        an1, an2 = sorted(annees)
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee between {an1} and {an2}")
    elif len(annees) == 1:
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee = {annees[0]}")
    return " AND ".join(clauses)


def extraire_reponse_sql(select_pred, where_clause):
    sql = f"SELECT {select_pred} FROM films WHERE {where_clause};"
    try:
        result = pd.read_sql_query(sql, conn)
        if result.empty:
            return sql, "Aucun résultat"
        return sql, result.head(3).to_string(index=False)
    except Exception as e:
        return sql, f"Erreur SQL : {e}"


def extraire_labels(sql):
    select = re.search(r'select (.+?) from', sql, re.IGNORECASE)
    return select.group(1).strip().lower() if select else "UNKNOWN"


select_X, select_y = [], []
for entry in data_train:
    select_label = extraire_labels(entry["sql"])
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for phrase in phrases:
        select_X.append(phrase)
        select_y.append(select_label)

pipeline_select = Pipeline([
    ("vec", CountVectorizer()),
    ("clf", Perceptron())
])
pipeline_select.fit(select_X, select_y)


requetes_test = [
    "Quels sont les films réalisés par Wes Anderson ?",
    "Donne-moi les films de genre comédie.",
    "Quels films sont sortis en 2011 ?",
    "Affiche les films avec Brad Pitt.",
    "Quels sont les films d’action ?",
    "Montre moi les film de horreur.",
    "c koi les film avec denzel washigton ?",
    "Les films reyalisés par Martin scorcesse stp.",
    "Donne les film ou ya Emma Stone.",
    "Est ce qu'il y a des film en 1983 ?",
    "Quels films de science-fiction ont été réalisés après 2010 ?",
    "Trouve les films où jouent à la fois Denzel Washington et Tom Cruise.",
    "Liste des films de Peter Jackson avec Gal Gadot.",
    "Quels sont les films sortis entre 2000 et 2010 de genre comédie ?",
    "Tous les films réalisés par Denis Villeneuve avec au moins un acteur féminin mentionné.",
    "Quels films ont le mot “Promise” dans le titre ?",
    "Trouve les films où Chris Hemsworth est acteur principal.",
    "Quels sont les films réalisés par des réalisateurs dont le prénom est Ron ?",
    "Quels films ont pour actrice Jennifer Lawrence ?",
    "Liste des films avec plus d’un acteur masculin."
]

# Récapitulatif
recap = []

for req in requetes_test:
    concepts = analyse_lexicale(req, valeurs_concepts)
    select_pred = pipeline_select.predict([req])[0]
    where_clause = generer_where_sql(concepts)
    sql, result = extraire_reponse_sql(select_pred, where_clause)

    recap.append({
        "Requête naturelle": req,
        "Requête SQL générée": sql,
        "Résultat": result
    })

# Affichage final
df_recap = pd.DataFrame(recap)
pd.set_option("display.max_colwidth", None)
print("\n🧾 Récapitulatif des requêtes testées :")
display(df_recap)



🧾 Récapitulatif des requêtes testées :


,Requête naturelle,Requête SQL générée,Résultat
0,Quels sont les films réalisés par Wes Anderson ?,SELECT titre FROM films WHERE realisateur = 'Wes Anderson';,titre\nThe Burning Truth Project\nThe Infinite Journey Saga\n The Frozen Echo Saga
1,Donne-moi les films de genre comédie.,SELECT titre FROM films WHERE genre = 'Comédie';,titre\nThe Burning Truth Project\nThe Infinite Secret Files\n The Hidden Shadow Saga
2,Quels films sont sortis en 2011 ?,SELECT titre FROM films WHERE annee = 2011;,titre\nThe Hidden Promise Game\n The Hidden Empire Saga\nThe Burning Shadow Saga
3,Affiche les films avec Brad Pitt.,SELECT titre FROM films WHERE ((acteur1 = 'Brad Pitt' or acteur2 = 'Brad Pitt' or acteur3 = 'Brad Pitt'));,titre\nThe Burning Truth Project\n The Lost Promise Saga\n The Golden Shadow Affair
4,Quels sont les films d’action ?,SELECT titre FROM films WHERE genre = 'Action';,titre\n The Infinite Shadow Story\nThe Last Journey Chronicles\nThe Silent Mission Protocol
5,Montre moi les film de horreur.,SELECT * FROM films WHERE genre = 'Horreur';,titre realisateur annee genre acteur1 acteur2 acteur3\nThe Hidden Promise Game Ron Howard 2011 Horreur Chris Hemsworth Natalie Portman Emily Blunt\n The Burning Echo Story Ron Howard 1986 Horreur Scarlett Johansson Robert De Niro Emma Stone\n The Frozen Echo Saga Wes Anderson 2016 Horreur Denzel Washington Johnny Depp Gal Gadot
6,c koi les film avec denzel washigton ?,SELECT titre FROM films WHERE ;,"Erreur SQL : Execution failed on sql 'SELECT titre FROM films WHERE ;': near "";"": syntax error"
7,Les films reyalisés par Martin scorcesse stp.,SELECT titre FROM films WHERE ;,"Erreur SQL : Execution failed on sql 'SELECT titre FROM films WHERE ;': near "";"": syntax error"
8,Donne les film ou ya Emma Stone.,SELECT titre FROM films WHERE ((acteur1 = 'Emma Stone' or acteur2 = 'Emma Stone' or acteur3 = 'Emma Stone'));,titre\nThe Burning Truth Project\n The Burning Echo Story\n The Lost Promise Story
9,Est ce qu'il y a des film en 1983 ?,"SELECT acteur1, acteur2, acteur3 FROM films WHERE annee = 1983;",acteur1 acteur2 acteur3\n Keanu Reeves Matt Damon Gal Gadot\n Mark Ruffalo Keanu Reeves Tom Hanks\nChris Hemsworth Chris Evans Brad Pitt


#Question bonus :

In [ ]:
import pandas as pd
import sqlite3
import re
import json
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron

# === Chargement
with open("queries_french_para.json", encoding="utf-8") as f:
    data_train = json.load(f)

df = pd.read_csv("base_films_500.csv")
conn = sqlite3.connect(":memory:")
df.to_sql("films", conn, index=False, if_exists="replace")

# === Lexique
valeurs_concepts = {
    'titre': df['titre'].dropna().unique().tolist(),
    'realisateur': df['realisateur'].dropna().unique().tolist(),
    'annee': df['annee'].dropna().astype(str).unique().tolist(),
    'genre': df['genre'].dropna().unique().tolist(),
    'acteur1': df['acteur1'].dropna().unique().tolist(),
    'acteur2': df['acteur2'].dropna().unique().tolist(),
    'acteur3': df['acteur3'].dropna().unique().tolist()
}

# === Analyse lexicale
def analyse_lexicale(requete, lexique):
    concepts_valeurs = []
    deja_vus = set()
    correspondances = []
    for concept, valeurs in lexique.items():
        for valeur in valeurs:
            if valeur.lower() in requete.lower():
                match = re.search(re.escape(valeur), requete, flags=re.IGNORECASE)
                if match:
                    correspondances.append((match.start(), valeur, concept))
    correspondances.sort()
    for _, valeur, concept in correspondances:
        if valeur.lower() not in deja_vus:
            deja_vus.add(valeur.lower())
            concept_label = "acteur" if concept.startswith("acteur") else concept
            concepts_valeurs.append((concept_label, valeur))
    return concepts_valeurs


def generer_where_sql(concepts_valeurs, requete_originale=None):
    clauses = []
    annees = []

    for concept, valeur in concepts_valeurs:
        if concept == "annee":
            annees.append(valeur)
        elif concept == "acteur":
            clause = f"((acteur1 = '{valeur}' or acteur2 = '{valeur}' or acteur3 = '{valeur}'))"
            clauses.append(clause)
        else:
            clause = f"{concept} = '{valeur}'"
            clauses.append(clause)


    if requete_originale:
        mots = re.findall(r"\b\w+\b", requete_originale)
        mots_utiles = [m for m in mots if len(m) > 3 and m.lower() not in ['quels', 'les', 'dans', 'avec', 'film', 'films', 'titre', 'mot', 'le']]
        valeurs_connues = [v.lower() for _, v in concepts_valeurs]
        for mot in mots_utiles:
            if mot.lower() not in valeurs_connues:
                clauses.append(f"titre LIKE '%{mot}%'")
                break

    if len(annees) == 2:
        an1, an2 = sorted(annees)
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee BETWEEN {an1} AND {an2}")
    elif len(annees) == 1:
        clauses = [c for c in clauses if not c.startswith("annee")]
        clauses.append(f"annee = {annees[0]}")

    return " AND ".join(clauses)

# === SQL
def executer_sql(select, where):
    sql = f"SELECT {select} FROM films WHERE {where};"
    try:
        result = pd.read_sql_query(sql, conn)
        if result.empty:
            return sql, "Aucun résultat"
        return sql, result.head(5).to_string(index=False)
    except Exception as e:
        return sql, f"Erreur : {e}"

# === SELECT (Perceptron)
def extraire_labels(sql):
    match = re.search(r'select (.+?) from', sql, re.IGNORECASE)
    return match.group(1).strip().lower() if match else "UNKNOWN"

select_X, select_y = [], []
for entry in data_train:
    label = extraire_labels(entry["sql"])
    phrases = [entry["french"]["query_french"]] + entry["french"]["paraphrase_french"]
    for p in phrases:
        select_X.append(p)
        select_y.append(label)

pipeline_select = Pipeline([
    ("vec", CountVectorizer()),
    ("clf", Perceptron())
])
pipeline_select.fit(select_X, select_y)

# === Requête BONUS à tester
requete = "Quels films ont le mot Promise dans le titre ?"

concepts = analyse_lexicale(requete, valeurs_concepts)
select = pipeline_select.predict([requete])[0]
where = generer_where_sql(concepts, requete_originale=requete)
sql, result = executer_sql(select, where)

# === Affichage
print("📝 Requête naturelle :", requete)
print("📄 SQL générée :", sql)
print("\n📊 Résultat :")
print(result)


📝 Requête naturelle : Quels films ont le mot Promise dans le titre ?
📄 SQL générée : SELECT realisateur FROM films WHERE titre LIKE '%Promise%';

📊 Résultat :
      realisateur
       Ron Howard
 Denis Villeneuve
  Stanley Kubrick
  Robert Zemeckis
Quentin Tarantino


Et tester aussi :

"Quels films commencent par The ?"

"Films dont le titre contient Shadow"

"Liste des films où on voit 'Revenge'"